In [65]:
import json
import ipysheet as sheet
# !pip install ipywidgets
# !pip install ipysheet

In [155]:
def get_team_hp(team, units):
    return sum(u['hp'] for u in units if u['team'] == team)

def get_distance(u1, u2):
    return abs(u1['x'] - u2['x'])

def find_nearest_enemy(me, units):
    md, nearest = 9001, None
    for u in units:
        if u['hp'] <= 0 or u['team'] == me['team']:
            continue
        dx = get_distance(me, u)
        if dx < md:
            dx = md
            nearest = u
    return nearest

def battle(team1, team2, px1, px2, dt=0.1, logging=False):
    units = []
    for i, px, team in [[0, px1, team1], [1, px2, team2]]:
        for x, unit_params in zip(px, team):
            u = {k:v for k, v in unit_params.items()}
            u['hp'] = u['maxHealth']
            u['team'] = i
            u['ready'] = 0
            u['x'] = x
            units.append(u)
    time = 0
    for _ in range(900):
        if get_team_hp(0, units) <= 0 or get_team_hp(1, units) <= 0:
            break
        for un in units:
            if un['hp'] <= 0 or un['ready'] > time:
                continue
            enemy = find_nearest_enemy(un, units)
            if not enemy:
                continue
            if get_distance(un, enemy) <= un['attackRange']:
                if logging:
                    print(f'T{time:0.1f}: {un["type"]} team {un["team"]} hits {enemy["type"]} for {un["attackDamage"]}')
                enemy['hp'] -= un['attackDamage']
                un['ready'] = time + un['attackCooldown']
            else:
                x = un['x']
                un['x'] = x + [1, -1][un['team']] * (un['maxSpeed'] * dt)
                if logging:
                    print(f'T{time:0.1f}: {un["type"]} team {un["team"]} moves from {x} to {un["x"]}')

        time += dt
    return time, units

In [134]:
with open("data/GiantsGatesUnitsParams.json") as datafile:
    data = json.load(datafile)

for k in data.keys():
    data[k]["type"] = k

param_names = [
    'cost',
    'maxSpeed',
    'maxHealth',
    'attackDamage',
    'attackCooldown',
    'attackRange']
unit_names = list(data.keys())
unit_sheet = sheet.sheet(
    columns=len(param_names),
    rows=len(unit_names),
    row_headers=unit_names,
    column_headers=param_names,
    numeric_format='0')
work_params = {}
for i, name in enumerate(unit_names):
    work_params[name] = sheet.row(i, [data[name][p] for p in param_names])

def get_unit_data(unit_type):
    res = {'type': unit_type}
    for i, p in enumerate(param_names):
        res[p] = work_params[unit_type].value[i]
    return res

unit_sheet

Sheet(cells=(Cell(column_end=5, column_start=0, row_end=0, row_start=0, squeeze_column=False, type='numeric', …

In [158]:
munchkin = get_unit_data('munchkin')
thrower = get_unit_data('thrower')
ogre = get_unit_data('ogre')
shaman = get_unit_data('shaman')
brawler = get_unit_data('brawler')
warlock = get_unit_data('warlock')
headhunter = get_unit_data('headhunter')

t, result = battle([munchkin], [munchkin], [0], [20], logging=False)

print(f'Time: {t}')
for un in result:
    print(f'{un["type"]} from team {un["team"]} health is {un["hp"]}')

Time: 5.699999999999996
munchkin from team 0 health is 0
munchkin from team 1 health is 10
